# SmartCampus Agentic RAG

Pipeline:

User Query
   ↓
Embedding-based Retrieval (FAISS)
   ↓
   
Top-k Chunk Selection
   ↓
Hallucination Guard
   - Retrieval Confidence
   - Keyword Coverage
   - Refusal Logic
   ↓
Strict Context-Constrained LLM
   ↓
Cited Answer / Summary / Quiz


In [1]:
!pip install -q langchain langchain-openai langchain-community langgraph
!pip install -q faiss-cpu pypdf tiktoken


In [2]:
import langchain
import faiss
import langgraph

print("All imports successful ✅")


All imports successful ✅


In [4]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [5]:
print("Key loaded ✅" if os.environ.get("OPENAI_API_KEY") else "Key missing ❌")


Key loaded ✅


In [6]:
from google.colab import files
uploaded = files.upload()


Saving Lab_6_Instructions_F24_updated.pdf to Lab_6_Instructions_F24_updated (1).pdf


In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

pdf_file = "Lab_6_Instructions_F24_updated.pdf"

# 1) Load PDF
loader = PyPDFLoader(pdf_file)
docs = loader.load()
print("Pages loaded:", len(docs))

# 2) Chunk
splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=150)
chunks = splitter.split_documents(docs)
print("Chunks created:", len(chunks))

# 3) Embeddings + vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(chunks, embeddings)

print("Vectorstore ready ✅")


Pages loaded: 9
Chunks created: 16
Vectorstore ready ✅


In [8]:
def show_retrieval(query, k=4):
    docs = vectorstore.similarity_search(query, k=k)
    print(f"Top {k} retrieved chunks for: {query}\n")
    for i, d in enumerate(docs, 1):
        page = d.metadata.get("page", "?")
        source = d.metadata.get("source", "pdf")
        print(f"--- Chunk {i} | page {page} | source {source} ---")
        print(d.page_content[:800])  # preview
        print()

show_retrieval("What are the deliverables for Lab 6?", k=4)



Top 4 retrieved chunks for: What are the deliverables for Lab 6?

--- Chunk 1 | page 0 | source Lab_6_Instructions_F24_updated.pdf ---
1. Follow the lab lecture about JavaDoc and unit tests then continue from step 2. 
2. Create a new project in Android Studio named ListyCity.
3. Create a new java class named City com.example.listycity. This class shall have two 
variables city and province. Create getters of these variables.
public class City  {
   private String city;
   private String province;
   City(String city, String province){
       this.city = city;
       this.province = province;
   }
   String getCityName(){
       return this.city;
   }
   String getProvinceName(){
       return this.province;
   }
}
4. Add Javadoc comments for City, its variables and functions. e.g.
/**  
* This is a class that defines a City.
*/ 
public class City {....}
5. Create a new class CityList under com.example.listycity. Now we wi

--- Chunk 2 | page 4 | source Lab_6_Instructions_F24_updated.pd

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

def rag_answer(question, k=4):
    docs = vectorstore.similarity_search(question, k=k)
    context = "\n\n---\n\n".join(
        [f"(page {d.metadata.get('page','?')})\n{d.page_content}" for d in docs]
    )
    messages = [
        SystemMessage(content="Answer using ONLY the provided context. If the context is missing details, say what is missing."),
        HumanMessage(content=f"CONTEXT:\n{context}\n\nQUESTION:\n{question}")
    ]
    return llm.invoke(messages).content

print(rag_answer("List the Lab 6 deliverables and any submission instructions."))


The provided context does not include specific details about the Lab 6 deliverables or submission instructions.


In [10]:
print(rag_answer("What classes and tests am I expected to implement in Lab 6? Be specific.", k=6))


In Lab 6, you are expected to implement the following classes and tests:

1. **Classes:**
   - `City`: This class should have two private variables (`city` and `province`), a constructor, and getter methods for these variables. You should also add Javadoc comments for this class and its methods.
   - `CityList`: This class is mentioned but not detailed in the provided context. You will need to implement this class and its methods, which are implied to manage a list of `City` objects.

2. **Tests:**
   - `CityListTest`: This test class should contain unit tests for the functionalities of the `CityList` class. You need to implement the following test methods:
     - `testAdd()`: This test should check if a city can be added to the `CityList` and verify the size of the list and the presence of the city.
     - `testAddException()`: This test should check if an `IllegalArgumentException` is thrown when attempting to add a city that already exists in the list.

Additionally, you are expecte

In [11]:
print(rag_answer("What should the CityListTest class contain and what testing approach does the lab describe?", k=6))


The CityListTest class should contain unit tests for the functionalities of the CityList class. Specifically, it should include tests for the add() method, which verifies that city objects can be successfully added to the CityList. The class should also have private methods for creating mock city objects and adding them to the cityList.

The testing approach described in the lab is test-driven development (TDD). This involves writing all the tests first, which should initially fail due to the lack of implementation of the CityList class and its methods. After writing the tests, the implementation of the CityList class and its methods can be provided, allowing the tests to pass.


In [12]:
import re

def find_lines(pattern: str):
    hits = []
    for d in docs:  # 'docs' is pages loaded earlier
        page = d.metadata.get("page", "?")
        lines = d.page_content.splitlines()
        for ln in lines:
            if re.search(pattern, ln, flags=re.IGNORECASE):
                hits.append((page, ln.strip()))
    return hits

hits = find_lines(r"\btest\b|CityListTest|junit|assert|@Test|testAdd|Exception")
print("Hits found:", len(hits))
for h in hits[:40]:
    print(f"(page {h[0]}) {h[1]}")


Hits found: 37
(page 0) comments for this class and also test this class by writing unit tests.
(page 0) Exception. Here we also have written JavaDoc comments with @param tag.
(page 1) throw new IllegalArgumentException();
(page 2) throw new IllegalArgumentException();
(page 4) 18. Now let’s move to writing unit tests. We will use junit 5 for this lab because it has more
(page 4) modern testing features. By default junit 4 is included in every project. To use junit 5
(page 4) testImplementation 'org.junit.jupiter:junit-jupiter-api:5.0.1'
(page 4) testRuntimeOnly 'org.junit.jupiter:junit-jupiter-engine:5.0.1'
(page 4) Junit-jupiter-api JUnit Jupiter API for writing tests and extensions.
(page 4) Junit-jupiter-engine JUnit Jupiter test engine implementation, only required at
(page 4) 19. Under com.example.listycity(test) folder, create a new class CityListTest to test the
(page 4) import org.junit.jupiter.api.Test;
(page 4) import static org.junit.jupiter.api.Assertions.*;
(page 4) class

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

def rag_answer_cited(question, k=10):
    docs_r = vectorstore.similarity_search(question, k=k)
    context_blocks = []
    for d in docs_r:
        page = d.metadata.get("page","?")
        context_blocks.append(f"[page {page}]\n{d.page_content}")
    context = "\n\n---\n\n".join(context_blocks)

    messages = [
        SystemMessage(content=
            "Answer using ONLY the provided context. "
            "Every requirement you mention must include (page X). "
            "If something isn't stated, say it isn't stated."
        ),
        HumanMessage(content=f"CONTEXT:\n{context}\n\nQUESTION:\n{question}")
    ]
    return llm.invoke(messages).content

print(rag_answer_cited("What exactly does the lab say CityListTest must include and what is the testing approach?", k=10))


The lab states that the CityListTest class must include tests for the functionalities of the CityList class, specifically mentioning the following requirements:

1. Write a test for the add() method in the CityList class (page 4).
2. Write a test method to check if an IllegalArgumentException is thrown when adding a city that already exists in the list (page 5).
3. Write another test method to test the behavior of the getCities() method in the CityList class (page 5).
4. Implement all the tests first, which must fail initially as there is no implementation of the CityList class and its methods (page 6).
5. After implementing the CityList class and its methods, the tests should pass (page 6).

The testing approach mentioned is called test-driven development, where tests are written before the actual implementation, ensuring that the implementation meets the specified requirements (page 6).


In [14]:
def classify_intent_simple(text: str) -> str:
    t = text.lower()
    if "quiz" in t or "test me" in t:
        return "quiz"
    if "summarize" in t or "summary" in t:
        return "summary"
    return "qa"

def quiz_from_pdf(topic: str, n=6):
    docs_r = vectorstore.similarity_search(topic, k=10)
    context = "\n\n---\n\n".join([f"[page {d.metadata.get('page','?')}]\n{d.page_content}" for d in docs_r])
    msgs = [
        SystemMessage(content=f"Create a {n}-question quiz from the context. Include answers at the end. Use only context."),
        HumanMessage(content=f"CONTEXT:\n{context}\n\nTOPIC:\n{topic}")
    ]
    return llm.invoke(msgs).content

def summarize_from_pdf(topic: str):
    return rag_answer_cited("Summarize: " + topic, k=10)

def agent_run(user_text: str):
    intent = classify_intent_simple(user_text)
    if intent == "summary":
        return summarize_from_pdf(user_text)
    if intent == "quiz":
        return quiz_from_pdf(user_text)
    return rag_answer_cited(user_text, k=10)

print(agent_run("Summarize what Lab 6 wants me to build."))
print("\n" + "="*80 + "\n")
print(agent_run("Make a quiz on the required Lab 6 steps and testing approach."))


Lab 6 wants you to build an Android project named ListyCity that includes the following components:

1. A Java class named City with two variables (city and province), along with their getters and Javadoc comments (page 0).
2. A Java class named CityList that maintains a list of City objects, includes methods to add cities, and has appropriate Javadoc comments (page 0).
3. Unit tests for the CityList class using JUnit 5, including tests for adding cities, retrieving the list of cities, and handling exceptions when adding duplicate cities (pages 4, 5, 6).
4. The implementation should follow test-driven development, where tests are written before the actual implementation of the CityList class (page 6). 

Additionally, you need to ensure that the necessary JUnit 5 dependencies are included in the Gradle file and configure the project to support lambda expressions if needed (pages 4, 5).


### Quiz on Lab 6 Steps and Testing Approach

1. **What testing framework is used in this lab for wr

In [15]:
def rag_answer_strict(question, k=10):
    docs_r = vectorstore.similarity_search(question, k=k)

    # Collect allowed pages from retrieved docs
    allowed_pages = sorted({d.metadata.get("page","?") for d in docs_r})
    allowed_pages_str = ", ".join([str(p) for p in allowed_pages])

    context_blocks = []
    for d in docs_r:
        page = d.metadata.get("page","?")
        context_blocks.append(f"[page {page}]\n{d.page_content}")
    context = "\n\n---\n\n".join(context_blocks)

    messages = [
        SystemMessage(content=
            "You MUST answer using ONLY the provided context.\n"
            f"You may ONLY cite these pages: {allowed_pages_str}.\n"
            "For each requirement/claim, add a citation like (page X).\n"
            "If the answer is not fully supported, say: 'Not stated in the provided pages.'"
        ),
        HumanMessage(content=f"CONTEXT:\n{context}\n\nQUESTION:\n{question}")
    ]
    return llm.invoke(messages).content


In [16]:
print(rag_answer_strict("List every unit test method the lab explicitly mentions for CityListTest and what each test checks.", k=12))


The lab explicitly mentions the following unit test methods for the CityListTest class:

1. **testAdd()**: This test checks if a city can be successfully added to the CityList. It verifies that the size of the city list increases by one after adding a city and that the added city is contained in the list (page 5).

2. **testAddException()**: This test checks if an `IllegalArgumentException` is thrown when attempting to add a city that already exists in the CityList (page 5).

3. **testGetCities()**: This test checks if the first city in the cityList (retrieved by `cityList.getCities().get(0)`) is the same as the city returned by the mockCity() method (page 5). 

4. **CityListTest class completion**: The lab mentions completing the CityListTest class, implying that additional tests may be added, but specific tests beyond those listed are not detailed (page 7). 

These are the tests explicitly mentioned in the provided context.


In [17]:
def summarize_from_pdf(topic: str):
    return rag_answer_strict("Summarize (exam-style bullets): " + topic, k=12)

def agent_run(user_text: str):
    intent = classify_intent_simple(user_text)
    if intent == "summary":
        return summarize_from_pdf(user_text)
    if intent == "quiz":
        return quiz_from_pdf(user_text)
    return rag_answer_strict(user_text, k=12)


In [18]:
print(agent_run("Summarize what Lab 6 wants me to build."))
print("\n" + "="*80 + "\n")
print(agent_run("Make a quiz on the required Lab 6 steps and testing approach."))


- Create a new Android project named ListyCity (page 0).
- Develop a Java class named City with two variables: city and province, including getters for these variables (page 0).
- Add Javadoc comments for the City class, its variables, and functions (page 0).
- Create a new class CityList to manage a list of City objects, including Javadoc comments for this class (page 0).
- Implement a list to hold City objects and a method to add City objects to this list, ensuring to throw an exception if a city already exists (page 7).
- Write unit tests for the CityList class using JUnit 5, including tests for adding cities and handling exceptions (pages 4, 5, 6).
- Ensure tests are written before the implementation of the CityList class, following test-driven development (page 6).
- Generate JavaDoc documentation from the comments in the code (page 3).


### Quiz on Lab 6 Steps and Testing Approach

1. **What testing framework is used in this lab for writing unit tests?**
   - A) JUnit 4
   - B) 

In [19]:
def quiz_from_pdf(topic: str, n=6):
    docs_r = vectorstore.similarity_search(topic, k=12)
    allowed_pages = sorted({d.metadata.get("page","?") for d in docs_r})
    allowed_pages_str = ", ".join([str(p) for p in allowed_pages])

    context = "\n\n---\n\n".join(
        [f"[page {d.metadata.get('page','?')}]\n{d.page_content}" for d in docs_r]
    )

    msgs = [
        SystemMessage(content=
            f"Create a {n}-question quiz from the context. Include answers.\n"
            f"ONLY use the provided context. ONLY cite these pages: {allowed_pages_str}.\n"
            "After EVERY answer, add a citation like (page X).\n"
            "Do NOT infer extra behavior. If something isn't explicitly stated, answer: 'Not stated in the provided pages.'"
        ),
        HumanMessage(content=f"CONTEXT:\n{context}\n\nTOPIC:\n{topic}")
    ]
    return llm.invoke(msgs).content


In [20]:
print(agent_run("Make a quiz on the required Lab 6 steps and testing approach."))


**Quiz on Lab 6 Steps and Testing Approach**

1. What testing framework is used for the lab, and what must be added to the Gradle file to use it?
   - **Answer:** JUnit 5 is used, and the following must be added to the Gradle file: 
     ```
     testImplementation 'org.junit.jupiter:junit-jupiter-api:5.0.1' 
     testRuntimeOnly 'org.junit.jupiter:junit-jupiter-engine:5.0.1'
     ``` (page 4)

2. What is the purpose of the `mockCityList()` method in the `CityListTest` class?
   - **Answer:** The `mockCityList()` method creates a new `CityList` object and adds a mock city to it. (page 7)

3. What should be done before running the tests if you encounter the error “Lambda expressions not supported at this language level”?
   - **Answer:** Change the Java language to version 1.8 by going to File → Project Structure. (page 5)

4. What is the expected behavior of the `testAddException()` method in the `CityListTest` class?
   - **Answer:** It checks if an `IllegalArgumentException` is throw

In [21]:
import math
from typing import List, Tuple, Dict, Any

def retrieve_with_scores(query: str, k: int = 8) -> List[Tuple[Any, float]]:
    """
    Returns list of (Document, score).
    For LangChain FAISS, 'score' is typically a distance (lower is better).
    """
    return vectorstore.similarity_search_with_score(query, k=k)

def score_to_confidence(score: float) -> float:
    """
    Convert a distance-like score into a 0-1 confidence proxy.
    Lower distance => higher confidence.
    This mapping is monotonic and stable.
    """
    return 1.0 / (1.0 + float(score))


In [22]:
from langchain_core.messages import SystemMessage, HumanMessage

def rag_answer_guarded(question: str, k: int = 8, min_conf: float = 0.20, min_cov: float = 0.20):
    retrieved = retrieve_with_scores(question, k=k)
    if not retrieved:
        return {"answer": "I couldn't retrieve any relevant context from the document.",
                "confidence": 0.0, "coverage": 0.0, "pages": [], "top_matches": []}

    top_matches = []
    pages_set = set()
    context_blocks = []

    for doc, score in retrieved:
        page = doc.metadata.get("page", "?")
        pages_set.add(page)
        conf = score_to_confidence(score)

        top_matches.append({
            "page": page,
            "score": float(score),
            "confidence": conf,
            "preview": doc.page_content[:180].replace("\n", " ")
        })
        context_blocks.append(f"[page {page}]\n{doc.page_content}")

    # Use avg top-3 confidence (more stable)
    overall_conf = sum(m["confidence"] for m in top_matches[:3]) / min(3, len(top_matches))
    allowed_pages = sorted(pages_set, key=lambda x: (str(x)))

    context = "\n\n---\n\n".join(context_blocks)
    cov = keyword_coverage(question, context)

    # Strong refusal if either retrieval is weak OR keywords aren't present
    if overall_conf < min_conf or cov < min_cov:
        return {
            "answer": (
                "I’m not confident the PDF supports that question.\n"
                f"- retrieval confidence={overall_conf:.2f}\n"
                f"- keyword coverage={cov:.2f}\n\n"
                "Try using exact terms from the lab (e.g., 'JUnit 5', 'CityListTest', 'useJUnitPlatform', "
                "'getCities', 'testAddException') or ask a more specific question."
            ),
            "confidence": overall_conf,
            "coverage": cov,
            "pages": allowed_pages,
            "top_matches": top_matches
        }

    allowed_pages_str = ", ".join([str(p) for p in allowed_pages])

    msgs = [
        SystemMessage(content=
            "You MUST answer using ONLY the provided context.\n"
            f"You may ONLY cite these pages: {allowed_pages_str}.\n"
            "For each concrete claim/requirement, add a citation like (page X).\n"
            "If something is not stated in the provided pages, say so."
        ),
        HumanMessage(content=f"CONTEXT:\n{context}\n\nQUESTION:\n{question}")
    ]

    ans = llm.invoke(msgs).content
    return {
        "answer": ans,
        "confidence": overall_conf,
        "coverage": cov,
        "pages": allowed_pages,
        "top_matches": top_matches
    }


In [23]:
def print_guarded_result(result, show_matches: int = 5):
    print("=== Answer ===")
    print(result["answer"])
    print("\n=== Guard ===")
    print(f"Confidence (proxy): {result.get('confidence',0):.2f}")
    print(f"Keyword coverage:   {result.get('coverage',0):.2f}")
    print(f"Pages retrieved:    {result.get('pages',[])}")
    print("\n=== Top retrieved chunks ===")
    for m in result.get("top_matches", [])[:show_matches]:
        print(f"- page {m['page']} | score {m['score']:.4f} | conf {m['confidence']:.2f} | {m['preview']}…")



In [27]:
res = rag_answer_guarded("What JUnit 5 dependencies do I add and where?", k=8, min_conf=0.20)
print_guarded_result(res)


=== Answer ===
You need to add the following JUnit 5 dependencies under the dependencies section in the app Gradle (build.gradle(Module:app)) file:

```groovy
testImplementation 'org.junit.jupiter:junit-jupiter-api:5.0.1' 
testRuntimeOnly 'org.junit.jupiter:junit-jupiter-engine:5.0.1'
```

After adding these lines, you should sync the project (page 4).

=== Guard ===
Confidence (proxy): 0.50
Keyword coverage:   1.00
Pages retrieved:    [0, 3, 4, 5, 6, 8]

=== Top retrieved chunks ===
- page 4 | score 0.8975 | conf 0.53 | 18. Now let’s move to writing unit tests. We will use junit 5 for this lab because it has more  modern testing features. By default junit 4 is included in every project. To use jun…
- page 8 | score 1.0457 | conf 0.49 | 29. Here is the gradle level dependencies…
- page 5 | score 1.1287 | conf 0.47 | assertEquals() and assertTrue(). We need to add @Test before any test method to  identify it as a junit test. @Test void testAdd() {    CityList cityList = mockCityList(); 

In [28]:
res2 = rag_answer_guarded("What is the best neural network architecture for this lab?", k=8, min_conf=0.20)
print_guarded_result(res2)


=== Answer ===
I’m not confident the PDF supports that question.
- retrieval confidence=0.38
- keyword coverage=0.00

Try using exact terms from the lab (e.g., 'JUnit 5', 'CityListTest', 'useJUnitPlatform', 'getCities', 'testAddException') or ask a more specific question.

=== Guard ===
Confidence (proxy): 0.38
Keyword coverage:   0.00
Pages retrieved:    [0, 1, 4, 5, 6, 7, 8]

=== Top retrieved chunks ===
- page 4 | score 1.6275 | conf 0.38 | 18. Now let’s move to writing unit tests. We will use junit 5 for this lab because it has more  modern testing features. By default junit 4 is included in every project. To use jun…
- page 0 | score 1.6645 | conf 0.38 | 1. Follow the lab lecture about JavaDoc and unit tests then continue from step 2.  2. Create a new project in Android Studio named ListyCity. 3. Create a new java class named City …
- page 6 | score 1.6780 | conf 0.37 | implement all the tests first and they must fail as there is no implementation of CityList  class and its method

In [26]:
import re

STOPWORDS = {
    "the","a","an","and","or","to","of","in","for","is","are","was","were","be","been",
    "this","that","it","as","on","with","by","at","from","what","which","when","where",
    "how","do","does","did","best"
}

def extract_keywords(text: str):
    words = re.findall(r"[a-zA-Z]{4,}", text.lower())  # 4+ letters
    return [w for w in words if w not in STOPWORDS]

def keyword_coverage(question: str, context: str) -> float:
    q_words = set(extract_keywords(question))
    if not q_words:
        return 0.0
    ctx = context.lower()
    hits = sum(1 for w in q_words if w in ctx)
    return hits / len(q_words)


In [29]:
from typing import Dict, Any

def agent_run_guarded(user_text: str, k: int = 12) -> Dict[str, Any]:
    intent = classify_intent_simple(user_text)

    if intent == "quiz":
        # Guard quiz topic itself first (answerability)
        guard = rag_answer_guarded(user_text, k=min(k, 12))
        # If guard refuses, return that refusal and don't generate quiz
        if guard["coverage"] < 0.20:
            guard["mode"] = "quiz_refused"
            return guard

        # otherwise generate quiz with citations
        quiz_text = quiz_from_pdf(user_text, n=6)
        return {
            "mode": "quiz",
            "answer": quiz_text,
            "confidence": guard["confidence"],
            "coverage": guard["coverage"],
            "pages": guard["pages"],
            "top_matches": guard["top_matches"]
        }

    # summary + qa both use guarded answering
    if intent == "summary":
        out = rag_answer_guarded("Summarize (exam-style bullets): " + user_text, k=min(k, 16))
        out["mode"] = "summary"
        return out

    out = rag_answer_guarded(user_text, k=min(k, 16))
    out["mode"] = "qa"
    return out


def print_agent_result(res: Dict[str, Any], show_matches: int = 4):
    print(f"=== Mode: {res.get('mode')} ===")
    print(res["answer"])
    print("\n=== Guard ===")
    print(f"Confidence: {res.get('confidence',0):.2f}")
    print(f"Coverage:   {res.get('coverage',0):.2f}")
    print(f"Pages:      {res.get('pages',[])}")
    print("\n=== Top matches ===")
    for m in res.get("top_matches", [])[:show_matches]:
        print(f"- page {m['page']} | score {m['score']:.4f} | conf {m['confidence']:.2f} | {m['preview']}…")


In [30]:
print_agent_result(agent_run_guarded("Summarize what Lab 6 wants me to build."))
print("\n" + "="*90 + "\n")
print_agent_result(agent_run_guarded("What JUnit 5 dependencies do I add and where?"))
print("\n" + "="*90 + "\n")
print_agent_result(agent_run_guarded("Make a quiz on CityListTest requirements."))


=== Mode: summary ===
- Create a new Android project named ListyCity (page 0).
- Develop a Java class named City with two variables: city and province, including getters for these variables (page 0).
- Add JavaDoc comments for the City class, its variables, and its functions (page 0).
- Create a new class named CityList to manage a list of City objects, including JavaDoc comments for this class (page 0, page 7).
- Implement a list to hold City objects within the CityList class (page 7).
- Implement a method in CityList to add City objects, ensuring it throws an exception if a city already exists (page 7).
- Write unit tests for the CityList class using JUnit 5, including tests for adding cities and handling exceptions (page 4, page 5).
- Use test-driven development by initially writing tests that fail due to the lack of implementation in CityList (page 6).
- Generate JavaDoc documentation from the comments in the code (page 3).

=== Guard ===
Confidence: 0.43
Coverage:   0.33
Pages:   

In [31]:
print_agent_result(agent_run_guarded("Make a quiz on neural networks for this lab."))







=== Mode: quiz_refused ===
I’m not confident the PDF supports that question.
- retrieval confidence=0.41
- keyword coverage=0.00

Try using exact terms from the lab (e.g., 'JUnit 5', 'CityListTest', 'useJUnitPlatform', 'getCities', 'testAddException') or ask a more specific question.

=== Guard ===
Confidence: 0.41
Coverage:   0.00
Pages:      [0, 3, 4, 5, 6, 7, 8]

=== Top matches ===
- page 4 | score 1.4357 | conf 0.41 | 18. Now let’s move to writing unit tests. We will use junit 5 for this lab because it has more  modern testing features. By default junit 4 is included in every project. To use jun…
- page 0 | score 1.4407 | conf 0.41 | 1. Follow the lab lecture about JavaDoc and unit tests then continue from step 2.  2. Create a new project in Android Studio named ListyCity. 3. Create a new java class named City …
- page 7 | score 1.5246 | conf 0.40 | 28. Complete CityListTest class…
- page 6 | score 1.5591 | conf 0.39 | implement all the tests first and they must fail as there is n

In [32]:
def guard_label(conf, cov):
    if cov >= 0.5 and conf >= 0.4:
        return "✅ High"
    if cov >= 0.2 and conf >= 0.2:
        return "⚠️ Medium"
    return "❌ Low"

def print_agent_result(res, show_matches=4):
    label = guard_label(res.get("confidence",0), res.get("coverage",0))
    print(f"=== Mode: {res.get('mode')} | Guard: {label} ===\n")
    print(res["answer"])
    print("\n=== Guard details ===")
    print(f"Confidence: {res.get('confidence',0):.2f}")
    print(f"Coverage:   {res.get('coverage',0):.2f}")
    print(f"Pages:      {res.get('pages',[])}")
    print("\n=== Top matches ===")
    for m in res.get("top_matches", [])[:show_matches]:
        print(f"- page {m['page']} | score {m['score']:.4f} | conf {m['confidence']:.2f} | {m['preview']}…")
